In [1]:
#####----README-----#######

#### STEP 1! GENERATE EXTERNAL VALIDATION SETS FOR ENRICHMENT PLOTS####
### RUN ALL THE CODE BLOCK COMBINATIONS BELOW TO MAKE ALL 3 EXTERNAL VALIDATION SETS####
### MAKE SURE TO RESTART THE KERNEL BEFORE EACH RUN ####


###Make randomNegatives: #1, #2, #3, #4
###Make restrictedNegatives: #1, #2, #3, #5, #6
###Make diverseNegatives: #1, #2, #3, #5, #7, #8, #9

In [1]:
import os
parent_dir = os.path.dirname(os.getcwd())
print(parent_dir)

/Users/eding/PycharmProjects/U24-ROBOKOP-Project-8-7-24


In [2]:
####1
import pandas as pd
import numpy as np
import random

unified_columns = [f"{i}" for i in range(604)]
filename = os.path.join(parent_dir, 'Step 1 Data Processing/ROBOKOP+DrugMechDB/ROBOKOP+DrugmechDB Data/ROBOMechDB Processed Triples.csv')
df=pd.read_csv(filename)
df.columns = ['0','drug_name','disease_name','protein_name']
df = df.drop(['0'],axis=1)

df_80 = df.sample(n=int(0.8*len(df)), random_state=42)  ##take 80% of these positive triples
df_80_indices = df_80.index.tolist()

all_indices = set(df.index) 
df_20_enrichment_indices = list(all_indices - set(df_80_indices))
df_20 = df.iloc[df_20_enrichment_indices] ###pocketing 20% of positive set out for validation

print(df_20)

####Find all unique drugs, diseases, and proteins in this 80% set. 
unique_triples_drug = sorted(list(set(df_20['drug_name'])))
unique_triples_disease = sorted(list(set(df_20['disease_name'])))
unique_triples_protein = sorted(list(set(df_20['protein_name'])))

pos_triples_dictionary = {}
pos_values = my_list = [1] * len(df)
pos_keys = [df.iloc[i,0] + " " + df.iloc[i,1]+ " " + df.iloc[i,2] for i in range(0,len(df))]
for key,value in zip(pos_keys,pos_values):
    pos_triples_dictionary[key] = value

           drug_name                                    disease_name  \
1           imatinib                           systemic mastocytosis   
8194     ruxolitinib                                        vitiligo   
8195       ezetimibe                                  sitosterolemia   
4        paracetamol                                            pain   
5        paracetamol                                            pain   
...              ...                                             ...   
8175     haloperidol                               tourette syndrome   
8178     haloperidol                               tourette syndrome   
8187     simvastatin  hyperlipidemia, familial combined, lpl related   
8189       diazoxide                         islet cell adenomatosis   
8190  norethisterone                        uterine corpus leiomyoma   

                                      protein_name  
1     kit proto-oncogene, receptor tyrosine kinase  
8194      homeodomain interac

In [3]:
####2 Importing created protein, disease, and drug vector dictionaries into here

def csvtodict(filename):
    vector_dict = {}
    df_vector_dict = pd.read_csv(filename, header = 0)
    keys = df_vector_dict.iloc[:,0].tolist()
    values = list(range(len(df_vector_dict)))
    for key,value in zip(keys,values):
        vector_dict[key] = value
    return vector_dict

protein_dict = csvtodict(os.path.join(parent_dir, 'Step 2 Data Embedding & Model Development/Vector Dictionaries/ROBOMechDB Protein Vector Dictionary.csv'))

disease_dict = csvtodict(os.path.join(parent_dir, 'Step 2 Data Embedding & Model Development/Vector Dictionaries/ROBOMechDB Disease Vector Dictionary.csv'))

drug_dict = csvtodict(os.path.join(parent_dir, 'Step 2 Data Embedding & Model Development/Vector Dictionaries/ROBOMechDB Drug Vector Dictionary.csv'))

protein_df = pd.read_csv(os.path.join(parent_dir, 'Step 2 Data Embedding & Model Development/Vector Dictionaries/ROBOMechDB Protein Vector Dictionary.csv'),header = 0)
disease_df = pd.read_csv(os.path.join(parent_dir, 'Step 2 Data Embedding & Model Development/Vector Dictionaries/ROBOMechDB Disease Vector Dictionary.csv'),header = 0)
drug_df = pd.read_csv(os.path.join(parent_dir, 'Step 2 Data Embedding & Model Development/Vector Dictionaries/ROBOMechDB Drug Vector Dictionary.csv'), header = 0)

In [4]:
###3 Building a 600 column vector array of all positive triples

positive_triples_vector_array = []
import itertools

for i in range(0,len(df_20)):
    try:
        drug_vector = drug_df.iloc[drug_dict[df_20.iloc[i,0]],1:201].tolist()
        disease_vector = disease_df.iloc[disease_dict[df_20.iloc[i,1]],1:201].tolist()
        protein_vector = protein_df.iloc[protein_dict[df_20.iloc[i,2]],1:201].tolist()
        
        drug_name = df_20.iloc[i,0]
        disease_name = df_20.iloc[i,1]
        protein_name = df_20.iloc[i,2]
        
        row = [[drug_name],[disease_name],[protein_name],drug_vector,disease_vector,protein_vector,[1]]
        merged = list(itertools.chain(*row))
        positive_triples_vector_array.append(merged)

    except KeyError:
        continue

positive_triples_dataframe = pd.DataFrame(positive_triples_vector_array)


In [6]:
print(positive_triples_dataframe)

                 0                                               1    \
0           imatinib                           systemic mastocytosis   
1        ruxolitinib                                        vitiligo   
2          ezetimibe                                  sitosterolemia   
3        paracetamol                                            pain   
4        paracetamol                                            pain   
...              ...                                             ...   
1834     haloperidol                               tourette syndrome   
1835     haloperidol                               tourette syndrome   
1836     simvastatin  hyperlipidemia, familial combined, lpl related   
1837       diazoxide                         islet cell adenomatosis   
1838  norethisterone                        uterine corpus leiomyoma   

                                               2        3        4    \
0     kit proto-oncogene, receptor tyrosine kinase  0.78078  0.

In [19]:
###4 Use random combinations of drug, disease, protein present in positive triples to form negatives. Create negative vector array

negative_triples_array = []
negative_triples_vector_array = []
temp = set()
i=0

random.seed(42)

while i < 10.5*len(positive_triples_dataframe):  ###create negative triples
    drug = random.sample(unique_triples_drug,k=1)[0]
    protein = random.sample(unique_triples_protein, k=1)[0]
    disease = random.sample(unique_triples_disease,k=1)[0]
    key = drug + " " + disease + " " + protein
    if (key in pos_triples_dictionary)or (key in temp):
        print('true')
        continue
    temp.add(key)
    negative_triples_array.append([drug, disease, protein])
    i+= 1 

df_negative_triples = pd.DataFrame(negative_triples_array)
    
for i in range(0,len(df_negative_triples)):
    try:
        drug_vector = drug_df.iloc[drug_dict[df_negative_triples.iloc[i,0]],1:201].tolist()
        disease_vector = disease_df.iloc[disease_dict[df_negative_triples.iloc[i,1]],1:201].tolist()
        protein_vector = protein_df.iloc[protein_dict[df_negative_triples.iloc[i,2]],1:201].tolist()
        
        drug_name = df_negative_triples.iloc[i,0]
        disease_name = df_negative_triples.iloc[i,1]
        protein_name = df_negative_triples.iloc[i,2]
        
        row = [[drug_name],[disease_name],[protein_name],drug_vector,disease_vector,protein_vector,[0]]
        merged = list(itertools.chain(*row))
        negative_triples_vector_array.append(merged)

    except KeyError:
        continue

negative_triples_dataframe = pd.DataFrame(negative_triples_vector_array)

df_randomNegatives = pd.concat([positive_triples_dataframe,negative_triples_dataframe],axis=0,ignore_index=True)
df_randomNegatives.columns = unified_columns
print(df_randomNegatives.columns)



df_randomNegatives.to_csv(os.path.join(parent_dir, 'Step 3 External Validation/Step 3 External Validation Datasets/randomNegatives Step 3 External Validation Set.csv'), index=False)


true
true
true
Index(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9',
       ...
       '594', '595', '596', '597', '598', '599', '600', '601', '602', '603'],
      dtype='object', length=604)


In [20]:
print(df_randomNegatives)

                        0                                       1  \
0                imatinib                   systemic mastocytosis   
1             ruxolitinib                                vitiligo   
2               ezetimibe                          sitosterolemia   
3             paracetamol                                    pain   
4             paracetamol                                    pain   
...                   ...                                     ...   
20208          nelfinavir                          atrophic vulva   
20209  methyltestosterone  disseminated intravascular coagulation   
20210            felbinac   central nervous system origin vertigo   
20211         amobarbital                             liposarcoma   
20212         propranolol               arteriosclerosis disorder   

                                                       2         3         4  \
0           kit proto-oncogene, receptor tyrosine kinase  0.780780  0.165620   
1          

In [5]:
###5 Compute distance thresholds of proteins, drugs, and diseases present
    
unique_protein_vectors = np.array(protein_df.iloc[:,1:201])
unique_disease_vectors = np.array(disease_df.iloc[:,1:201])
unique_drug_vectors = np.array(drug_df.iloc[:,1:201])

unique_protein_dict_names = np.array(protein_df.iloc[:,0])
unique_disease_dict_names = np.array(disease_df.iloc[:,0])
unique_drug_dict_names = np.array(drug_df.iloc[:,0])


def compute_threshold(vector_dict,n=0.5):
    distances = []
    for i in range((len(vector_dict)-1)):
        smallest_distance = 10000000
        for j in range(i+1, len(vector_dict)):
            distance = np.linalg.norm(vector_dict[i]-vector_dict[j])
            if distance < smallest_distance:
                smallest_distance = distance
        distances.append(smallest_distance)

    distances = np.array(distances)
    print(len(distances))
    print('min', np.min(distances))
    mean_distances = np.sum(distances)/len(distances)
    stdev_distance = np.std(distances)
    print('mean',mean_distances)
    print('stdev',stdev_distance)

    applicability_domain = mean_distances+(n*stdev_distance)

    return applicability_domain

drug_threshold = compute_threshold(unique_drug_vectors)
disease_threshold = compute_threshold(unique_disease_vectors)
protein_threshold = compute_threshold(unique_protein_vectors)

print(drug_threshold)
print(disease_threshold)
print(protein_threshold)

998
min 1.3490766481180738
mean 3.9296513978117704
stdev 1.1837539597778333
589
min 1.3756730714527872
mean 6.979102681442049
stdev 2.672071221220308
835
min 1.00340273053115
mean 6.690955269076238
stdev 4.105268395143029
4.521528377700687
8.315138292052202
8.743589466647753


In [12]:
###6 Negative triples by keeping two of of the three features of the triple constant, 
#and randomly selecting a drug/disease/triple (depending on circumstance) for the last feature, 
#and compare to similarity thresholds

###

#Let n = # of positive triples in dataset
#First select n/4 rows 

#we are now going to split these rows into 3 equal chunks. 
#the triples in the first chunk will have its protein parameter randomized, 
#the triples in the second chunk will have its disease parameter randomized, and so on

split_rows = np.array_split(positive_triples_dataframe, 3)

# Each part is now a separate DataFrame
drug_disease_x, drug_x_protein, x_disease_protein = split_rows[0], split_rows[1], split_rows[2]

drug_disease_x_negative_triples = []
drug_x_protein_negative_triples = []
x_disease_protein_negative_triples = []
temp = set()

for i in range(len(drug_disease_x)):
    reference = np.array(drug_disease_x.iloc[i, 403:603]) ###not including 401, including 601
    reference = [float(item) for item in reference]
    counter=0
    for j in range(len(unique_protein_vectors)):
        if (np.linalg.norm(reference-unique_protein_vectors[j]) < protein_threshold):
            triple_name = [drug_disease_x.iloc[i,0], drug_disease_x.iloc[i,1], unique_protein_dict_names[j]]
            new_combo = drug_disease_x.iloc[i,0] + " " + drug_disease_x.iloc[i,1] + " " + unique_protein_dict_names[j]
            if (new_combo in pos_triples_dictionary or new_combo in temp):
                continue
            temp.add(new_combo)
            triple_vector = list(itertools.chain(*[np.array(drug_disease_x.iloc[i,3:403]).tolist(), unique_protein_vectors[j].tolist()]))
            row = list(itertools.chain(*[triple_name,triple_vector,[0]]))
            drug_disease_x_negative_triples.append(row)
            counter+=1
            if counter >=20:
                break

for i in range(len(drug_x_protein)):
    reference = np.array(drug_x_protein.iloc[i, 203:403])
    reference = [float(item) for item in reference]
    counter=0
    for j in range(len(unique_disease_vectors)):
        if (np.linalg.norm(reference-unique_disease_vectors[j]) < disease_threshold):
            triple_name = [drug_x_protein.iloc[i,0], unique_disease_dict_names[j],drug_x_protein.iloc[i,2]]
            new_combo = drug_x_protein.iloc[i,0] + " " + unique_disease_dict_names[j] + " " + drug_x_protein.iloc[i,2]
            if (new_combo in pos_triples_dictionary or new_combo in temp):
                continue
            temp.add(new_combo)
            triple_vector = list(itertools.chain(*[np.array(drug_x_protein.iloc[i,3:203]).tolist(), unique_disease_vectors[j].tolist(),np.array(drug_x_protein.iloc[i,403:603]).tolist()]))
            row = list(itertools.chain(*[triple_name,triple_vector,[0]]))
            drug_x_protein_negative_triples.append(row)
            counter +=1
            if counter >=19:
                break

for i in range(len(x_disease_protein)):
    reference = np.array(x_disease_protein.iloc[i, 3:203])
    reference = [float(item) for item in reference]
    counter=0    
    for j in range(len(unique_drug_vectors)):
        if (np.linalg.norm(reference-unique_drug_vectors[j]) < drug_threshold):
            triple_name = [unique_drug_dict_names[j],x_disease_protein.iloc[i,1], x_disease_protein.iloc[i,2]]
            new_combo = unique_drug_dict_names[j] + " " + x_disease_protein.iloc[i,1] + " " + x_disease_protein.iloc[i,2]
            if (new_combo in pos_triples_dictionary or new_combo in temp):
                continue
            temp.add(new_combo)
            triple_vector = list(itertools.chain(*[unique_drug_vectors[j].tolist(),np.array(x_disease_protein.iloc[i,203:603]).tolist()]))
            row = list(itertools.chain(*[triple_name,triple_vector,[0]]))
            x_disease_protein_negative_triples.append(row)
            counter += 1
            if counter >=20:
                break
            
negative_df = pd.concat([pd.DataFrame(drug_disease_x_negative_triples),pd.DataFrame(drug_x_protein_negative_triples),pd.DataFrame(x_disease_protein_negative_triples)],axis=0,ignore_index=True)
df_restrictedNegs = pd.concat([positive_triples_dataframe,negative_df],axis=0)
df_restrictedNegs.columns = unified_columns
df_restrictedNegs.to_csv(os.path.join(parent_dir, 'Step 3 External Validation/Step 3 External Validation Datasets/restrictedNegatives Step 3 External Validation Set.csv'), index=False)

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


In [13]:
print(negative_df)

                       0                         1    \
0                 imatinib     systemic mastocytosis   
1                 imatinib     systemic mastocytosis   
2              ruxolitinib                  vitiligo   
3              ruxolitinib                  vitiligo   
4                ezetimibe            sitosterolemia   
...                    ...                       ...   
18425       levonorgestrel  uterine corpus leiomyoma   
18426  medroxyprogesterone  uterine corpus leiomyoma   
18427            mestranol  uterine corpus leiomyoma   
18428         norgestimate  uterine corpus leiomyoma   
18429         progesterone  uterine corpus leiomyoma   

                                                    2        3         4    \
0          met proto-oncogene, receptor tyrosine kinase  0.78078  0.165620   
1      src proto-oncogene, non-receptor tyrosine kinase  0.78078  0.165620   
2              homeodomain interacting protein kinase 2  0.87328  0.170770   
3              

In [10]:
###7 (ONLY RUN THIS WHEN MAKING DIVERSENEGATIVES) generate randomNegative Triples

negative_triples_array = []
negative_triples_vector_array = []
temp = set()
i=0

random.seed(42)

while i < 9.5*len(positive_triples_dataframe):  ###create negative triples 
    drug = random.sample(unique_triples_drug,k=1)[0]
    protein = random.sample(unique_triples_protein, k=1)[0]
    disease = random.sample(unique_triples_disease,k=1)[0]
    if (drug + " " + disease + " " + protein) in pos_triples_dictionary or (drug + " " + disease + " " + protein) in temp:
        continue
    temp.add(drug + " " + disease + " " + protein)
    negative_triples_array.append([drug, disease, protein])
    i+= 1 

df_negative_triples = pd.DataFrame(negative_triples_array)

for i in range(0,len(df_negative_triples)):
    try:
        drug_vector = drug_df.iloc[drug_dict[df_negative_triples.iloc[i,0]],1:201].tolist()
        disease_vector = disease_df.iloc[disease_dict[df_negative_triples.iloc[i,1]],1:201].tolist()
        protein_vector = protein_df.iloc[protein_dict[df_negative_triples.iloc[i,2]],1:201].tolist()
        
        drug_name = df_negative_triples.iloc[i,0]
        disease_name = df_negative_triples.iloc[i,1]
        protein_name = df_negative_triples.iloc[i,2]
        
        row = [[drug_name],[disease_name],[protein_name],drug_vector,disease_vector,protein_vector,[0]]
        merged = list(itertools.chain(*row))
        negative_triples_vector_array.append(merged)

    except KeyError:
        continue

rand_negative_triples_dataframe = pd.DataFrame(negative_triples_vector_array)



In [11]:
#(ONLY RUN THIS WHEN MAKING DIVERSENEGATIVES) ###
###8 generate restrictedNegative triples 

#we are now going to split these rows into 3 equal chunks. 
#the triples in the first chunk will have its protein parameter randomized, 
#the triples in the second chunk will have its disease parameter randomized, and so on

split_rows = np.array_split(positive_triples_dataframe, 3)

# Each part is now a separate DataFrame
drug_disease_x, drug_x_protein, x_disease_protein = split_rows[0], split_rows[1], split_rows[2]

drug_disease_x_negative_triples = []
drug_x_protein_negative_triples = []
x_disease_protein_negative_triples = []
temp = set()

for i in range(int(len(drug_disease_x)/1.3)): 
    reference = np.array(drug_disease_x.iloc[i, 403:603]) ###not including 401, including 601
    reference = [float(item) for item in reference]
    counter=0
    for j in range(len(unique_protein_vectors)):
        if counter >=1:
                break
        if (np.linalg.norm(reference-unique_protein_vectors[j]) < protein_threshold):
            triple_name = [drug_disease_x.iloc[i,0], drug_disease_x.iloc[i,1], unique_protein_dict_names[j]]
            new_combo = drug_disease_x.iloc[i,0] + " " + drug_disease_x.iloc[i,1] + " " + unique_protein_dict_names[j]
            if (new_combo in pos_triples_dictionary or new_combo in temp):
                continue
            temp.add(new_combo)
            triple_vector = list(itertools.chain(*[np.array(drug_disease_x.iloc[i,3:403]).tolist(), unique_protein_vectors[j].tolist()]))
            row = list(itertools.chain(*[triple_name,triple_vector,[0]]))
            drug_disease_x_negative_triples.append(row)
            counter+=1

for i in range(int(len(drug_x_protein)/1.25)):
    reference = np.array(drug_x_protein.iloc[i, 203:403])
    reference = [float(item) for item in reference]
    counter=0
    for j in range(len(unique_disease_vectors)):
        if counter >=2:
                break
        if (np.linalg.norm(reference-unique_disease_vectors[j]) < disease_threshold):
            triple_name = [drug_x_protein.iloc[i,0], unique_disease_dict_names[j],drug_x_protein.iloc[i,2]]
            new_combo = drug_x_protein.iloc[i,0] + " " + unique_disease_dict_names[j] + " " + drug_x_protein.iloc[i,2]
            if (new_combo in pos_triples_dictionary or new_combo in temp):
                continue
            temp.add(new_combo)
            triple_vector = list(itertools.chain(*[np.array(drug_x_protein.iloc[i,3:203]).tolist(), unique_disease_vectors[j].tolist(),np.array(drug_x_protein.iloc[i,403:603]).tolist()]))
            row = list(itertools.chain(*[triple_name,triple_vector,[0]]))
            drug_x_protein_negative_triples.append(row)
            counter +=1

for i in range(int(len(x_disease_protein)/1.3)):
    reference = np.array(x_disease_protein.iloc[i, 3:203])
    reference = [float(item) for item in reference]
    counter=0    
    for j in range(len(unique_drug_vectors)):
        if (np.linalg.norm(reference-unique_drug_vectors[j]) < drug_threshold):
            if counter >=1:
                break
            triple_name = [unique_drug_dict_names[j],x_disease_protein.iloc[i,1], x_disease_protein.iloc[i,2]]
            new_combo = unique_drug_dict_names[j] + " " + x_disease_protein.iloc[i,1] + " " + x_disease_protein.iloc[i,2]
            if (new_combo in pos_triples_dictionary or new_combo in temp):
                continue
            temp.add(new_combo)
            triple_vector = list(itertools.chain(*[unique_drug_vectors[j].tolist(),np.array(x_disease_protein.iloc[i,203:603]).tolist()]))
            row = list(itertools.chain(*[triple_name,triple_vector,[0]]))
            x_disease_protein_negative_triples.append(row)
            counter += 1

restricted_negative_triples_dataframe = pd.concat([pd.DataFrame(drug_disease_x_negative_triples),pd.DataFrame(drug_x_protein_negative_triples),pd.DataFrame(x_disease_protein_negative_triples)],axis=0,ignore_index=True)

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


In [12]:
####9. GENERATE EXTERNAL VALIDATION SET diverseNegatives####

restricted_negative_triples_dataframe.columns = unified_columns
rand_negative_triples_dataframe.columns = unified_columns
positive_triples_dataframe.columns = unified_columns

neg_total = pd.concat([restricted_negative_triples_dataframe,rand_negative_triples_dataframe],axis=0)
neg_total.columns = unified_columns
final_columns = ['0','1','2']
neg_total = neg_total.drop_duplicates(subset = final_columns).reset_index().drop('index', axis = 1)

df_diverseNegs = pd.concat([positive_triples_dataframe,neg_total],axis=0,ignore_index=True)

df_diverseNegs.to_csv(os.path.join(parent_dir, 'Step 3 External Validation/Step 3 External Validation Datasets/diverseNegatives Step 3 External Validation Set.csv'), index=False)

In [13]:
print(restricted_negative_triples_dataframe)
print(rand_negative_triples_dataframe)

                       0                        1  \
0               imatinib    systemic mastocytosis   
1            ruxolitinib                 vitiligo   
2              ezetimibe           sitosterolemia   
3            paracetamol                     pain   
4     methyltestosterone           cryptorchidism   
...                  ...                      ...   
1675          acebutolol  acute coronary syndrome   
1676        oxymetholone                   iritis   
1677           belotecan      ocular hypertension   
1678          acebutolol      ocular hypertension   
1679       carbinoxamine          pulmonary edema   

                                                    2         3         4  \
0        met proto-oncogene, receptor tyrosine kinase  0.780780  0.165620   
1            homeodomain interacting protein kinase 2  0.873280  0.170770   
2           atp binding cassette subfamily a member 1  0.456190  0.278480   
3                          prostaglandin e receptor 1  

In [1]:
####README#######

#### STEP 2. GENERATE INTERNAL VALIDATION SETS & MODEL TRAINING####

### RUN ALL THE CODE BLOCK COMBINATIONS BELOW####
### MAKE SURE TO RESTART THE KERNEL BEFORE EACH RUN ####

#represents the block number of the code block

#Complete Set 1. positive triples (w/o kmeans clustering) + randomNegative triples (without distanceThreshold) —> run #1, #2, #3
#Complete Set 2. positive triples (w/o kmeans clustering) + restrictedNegative triples (with distanceThreshold) —> run #1, #2, #4
#Complete Set 3. positive triples (w/ kmeans clustering) + randomNegative triples (without distanceThreshold) —> run #1, #2, #5
#Complete Set 4. positive triples (w/ kmeans clustering) + randomNegative triples (with distanceThreshold) —> run #1, #2, #6


In [1]:
#####1.

import pandas as pd
import os
parent_dir = os.path.dirname(os.getcwd())
print(parent_dir)


df_randNegs = pd.read_csv(os.path.join(parent_dir, 'Step 3 External Validation/Step 3 External Validation Datasets/randomNegatives Step 3 External Validation Set.csv'))
df_restrictedNegs = pd.read_csv(os.path.join(parent_dir, 'Step 3 External Validation/Step 3 External Validation Datasets/restrictedNegatives Step 3 External Validation Set.csv'))
df_diverseNegs = pd.read_csv(os.path.join(parent_dir, 'Step 3 External Validation/Step 3 External Validation Datasets/diverseNegatives Step 3 External Validation Set.csv'))

df_randNegs.drop(['0'],axis=1)
df_restrictedNegs.drop(['0'],axis=1)
df_diverseNegs.drop(['0'],axis=1)

unified_columns = [f"{i}" for i in range(604)]
filename = os.path.join(parent_dir, 'Step 1 Data Processing/ROBOKOP+DrugMechDB/ROBOKOP+DrugmechDB Data/ROBOMechDB Processed Triples.csv')
df=pd.read_csv(filename)
df = df.drop(['Unnamed: 0'],axis=1)
df.columns = ['0','1','2']

df_ext = pd.concat([df_randNegs.iloc[:,:3],df_restrictedNegs.iloc[:,:3],df_diverseNegs.iloc[:,:3],df],axis=0)

df_80 = df.sample(n=int(0.8*len(df)), random_state=42)  ##take 80% of these positive triples
df_80_indices = df_80.index.tolist()

pos_neg_triples_dictionary = {}
values = my_list = [1] * len(df_ext)
keys = [df_ext.iloc[i,0] + " " + df_ext.iloc[i,1]+ " " + df_ext.iloc[i,2] for i in range(0,len(df_ext))]
for key,value in zip(keys,values):
    pos_neg_triples_dictionary[key] = value


####Find all unique drugs, diseases, and proteins in this 80% set. 
unique_triples_drug = sorted(list(set(df_80['0'])))
unique_triples_disease =sorted(list(set(df_80['1'])))
unique_triples_protein = sorted(list(set(df_80['2'])))


/Users/eding/PycharmProjects/U24-ROBOKOP-Project-8-7-24


In [2]:
####2 Importing created protein, disease, and drug vector dictionaries into here

def csvtodict(filename):
    vector_dict = {}
    df_vector_dict = pd.read_csv(filename, header = 0)
    keys = df_vector_dict.iloc[:,0].tolist()
    values = list(range(len(df_vector_dict)))
    for key,value in zip(keys,values):
        vector_dict[key] = value
    return vector_dict


protein_dict = csvtodict(os.path.join(parent_dir,
                                      'Step 2 Data Embedding & Model Development/Vector Dictionaries/ROBOMechDB Protein Vector Dictionary.csv'))

disease_dict = csvtodict(os.path.join(parent_dir,
                                      'Step 2 Data Embedding & Model Development/Vector Dictionaries/ROBOMechDB Disease Vector Dictionary.csv'))

drug_dict = csvtodict(os.path.join(parent_dir,
                                   'Step 2 Data Embedding & Model Development/Vector Dictionaries/ROBOMechDB Drug Vector Dictionary.csv'))

protein_df = pd.read_csv(os.path.join(parent_dir,
                                      'Step 2 Data Embedding & Model Development/Vector Dictionaries/ROBOMechDB Protein Vector Dictionary.csv'),
                         header=0)
disease_df = pd.read_csv(os.path.join(parent_dir,
                                      'Step 2 Data Embedding & Model Development/Vector Dictionaries/ROBOMechDB Disease Vector Dictionary.csv'),
                         header=0)
drug_df = pd.read_csv(os.path.join(parent_dir,
                                   'Step 2 Data Embedding & Model Development/Vector Dictionaries/ROBOMechDB Drug Vector Dictionary.csv'),
                      header=0)

In [3]:
os.getcwd()

'/Users/eding/PycharmProjects/U24-ROBOKOP-Project-8-7-24/External Validation'

In [ ]:
###3 Complete Set 1 Development

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import auc, roc_curve
import tensorflow as tf
import pandas as pd
import numpy as np
import itertools
import random

random_seeds = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,42]

model_auc_values = []

for i in random_seeds:
    random.seed(i)
    positive_triples_vector_array = []
    for j in range(0,len(df_80)):
        try:
            drug_vector = drug_df.iloc[drug_dict[df_80.iloc[j,0]],1:201].tolist()
            disease_vector = disease_df.iloc[disease_dict[df_80.iloc[j,1]],1:201].tolist()
            protein_vector = protein_df.iloc[protein_dict[df_80.iloc[j,2]],1:201].tolist()
        
            drug_name = df_80.iloc[j,0]
            disease_name = df_80.iloc[j,1]
            protein_name = df_80.iloc[j,2]
        
            row = [[drug_name],[disease_name],[protein_name],drug_vector,disease_vector,protein_vector,[1]]
            merged = list(itertools.chain(*row))
            positive_triples_vector_array.append(merged)

        except KeyError:
            continue

    positive_triples_dataframe = pd.DataFrame(positive_triples_vector_array)

    negative_triples_array = []
    negative_triples_vector_array = []
    temp = set()
    j=0

    while j < int(1.1*len(positive_triples_dataframe)):  ###create negative triples
        drug = random.sample(unique_triples_drug,k=1)[0]
        protein = random.sample(unique_triples_protein, k=1)[0]
        disease = random.sample(unique_triples_disease,k=1)[0]
        if (drug + " " + disease + " " + protein) in pos_neg_triples_dictionary or (drug + " " + disease + " " + protein) in temp:
            continue
        temp.add(drug + " " + disease + " " + protein)
        negative_triples_array.append([drug, disease, protein])
        j+= 1 

    df_negative_triples = pd.DataFrame(negative_triples_array)
    j=0
    for j in range(0,len(df_negative_triples)):
        try:
            drug_vector = drug_df.iloc[drug_dict[df_negative_triples.iloc[j,0]],1:201].tolist()
            disease_vector = disease_df.iloc[disease_dict[df_negative_triples.iloc[j,1]],1:201].tolist()
            protein_vector = protein_df.iloc[protein_dict[df_negative_triples.iloc[j,2]],1:201].tolist()
        
            drug_name = df_negative_triples.iloc[j,0]
            disease_name = df_negative_triples.iloc[j,1]
            protein_name = df_negative_triples.iloc[j,2]
        
            row = [[drug_name],[disease_name],[protein_name],drug_vector,disease_vector,protein_vector,[0]]
            merged = list(itertools.chain(*row))
            negative_triples_vector_array.append(merged)

        except KeyError:
            continue

    negative_triples_dataframe = pd.DataFrame(negative_triples_vector_array)
    df = pd.concat([positive_triples_dataframe,negative_triples_dataframe],axis=0,ignore_index=True)
    df.columns = unified_columns
    test = pd.concat([df_diverseNegs, df],axis=0)
    col = ['0', '1', '2']
    print(len(test))
    rem = test.drop_duplicates(subset = col)
    print(len(rem))

# Assuming df is already defined and contains your dataset
    
# Assuming df is already defined and contains your dataset
    shuffled_df = df.sample(frac=1.0, random_state=i) 
    data = shuffled_df.iloc[:, 3:-1].values
    labels = shuffled_df.iloc[:, -1].values
    print(shuffled_df)
    num_folds = 5
    skf = StratifiedKFold(n_splits=num_folds, shuffle=True, random_state=i)

    all_predictions = []
    all_test_sets = []

    # Perform 5-fold cross-validation to test the accuracy of the model
    fold = 1
    auc_values = []

    tf.keras.utils.set_random_seed(i)

    for train_index, test_index in skf.split(data, labels):
        print(f"Fold: {fold}")
    
    # Split the data into training and test sets
        X_train, X_test = data[train_index], data[test_index]
        y_train, y_test = labels[train_index], labels[test_index]
        tf_train_set = tf.data.Dataset.from_tensor_slices((X_train,y_train))
        tf_test_set = tf.data.Dataset.from_tensor_slices((X_test,y_test))
        
        BATCH_SIZE = 32
        STEPS_PER_EPOCH = len(X_train)//BATCH_SIZE
        tf_train_set = tf_train_set.repeat().batch(BATCH_SIZE)
        tf_test_set = tf_test_set.batch(BATCH_SIZE)

    # Define the learning rate schedule
        lr_schedule = tf.keras.optimizers.schedules.InverseTimeDecay(
            0.001,
            decay_steps=STEPS_PER_EPOCH * 1000,
            decay_rate=1,
            staircase=False
        )
        
        
        def get_optimizer():
            return tf.keras.optimizers.Adam(lr_schedule)

        optimizer = get_optimizer()
        
        model = tf.keras.models.Sequential([
            tf.keras.Input(shape=(600,)),
            tf.keras.layers.Dense(1024, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)), 
            tf.keras.layers.Dropout(0.8),
            tf.keras.layers.Dense(512, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)),
            tf.keras.layers.Dropout(0.8),
            tf.keras.layers.Dense(256, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)),
            tf.keras.layers.Dropout(0.8),
            tf.keras.layers.Dense(1, activation='sigmoid')
        ])

        model.compile(optimizer=optimizer,
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

        # Implement early stopping
        early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
        
        # Train the model
        model.fit(tf_train_set,
                    steps_per_epoch=STEPS_PER_EPOCH, 
                    epochs=40,
                    validation_data = tf_test_set, 
                    callbacks=[early_stopping], 
                    verbose=0)
        
        predictions = model.predict(X_test)
        fpr,tpr, thresholds = roc_curve(y_test,predictions)
        roc_auc = auc(fpr, tpr)
        auc_values.append(roc_auc)
        print("AUROC for test data:", roc_auc)
        all_predictions.append(predictions)
        all_test_sets.append(y_test)
        
        ####Before each iteration ends, save each model###
        
        save_model= tf.keras.models.save_model(model, os.path.join(os.getcwd(), f'Classification Models/Complete Set 1 Models/ROBOMechDB Complete Set 1 Model Seed {i}.keras'))

        fold += 1
        

    fcv_mean_auc = np.mean(auc_values)
    fcv_std_auc = np.std(auc_values)
    fcv_min_auc = np.min(auc_values)
    fcv_max_auc = np.max(auc_values)
    print(fcv_mean_auc, fcv_min_auc,fcv_max_auc)
    model_auc_values.append([fcv_min_auc,fcv_max_auc,fcv_mean_auc,fcv_std_auc])

fcv_df = pd.DataFrame(model_auc_values, columns=['min_auc', 'max_auc', fcv_mean_auc,fcv_std_auc])

min_auc = np.min(fcv_df.iloc[:,0])
max_auc = np.max(fcv_df.iloc[:,1])
avg_auc = np.mean(fcv_df.iloc[:,2])
std_dev_auc = np.mean(fcv_df.iloc[:,3])

ensemble_stats = np.array([min_auc,max_auc,avg_auc,std_dev_auc])
ensemble_stats = ensemble_stats.reshape(1, -1)
ensemble_stats_df = pd.DataFrame(ensemble_stats, columns = ['min_auc', 'max_auc', 'avg_auc', 'std_dev_auc'])
ensemble_stats_df.to_csv(os.path.join(os.getcwd(), 'Classification Models/Complete Set 1 Models/complete_set_1_ensemble_stats.csv'))


34901
34901
                     0                              1  \
2245         meloxicam           rheumatoid arthritis   
6419        prednisone             nephrotic syndrome   
8541    dimenhydrinate                    tyrosinemia   
533          celecoxib                   dysmenorrhea   
3964         gefitinib  non-small cell lung carcinoma   
...                ...                            ...   
905        perospirone                  schizophrenia   
5192        prilocaine          premature ejaculation   
12172       stanozolol         vitamin b12 deficiency   
235        vincristine   acute lymphoblastic leukemia   
13349  incadronic acid            bacterial pneumonia   

                                                       2         3         4  \
2245               prostaglandin-endoperoxide synthase 2  0.610850 -0.219320   
6419                                             albumin  0.239800 -0.247750   
8541   potassium voltage-gated channel subfamily a me... -0.021

In [3]:
###4 Complete Set 2 Development

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import auc, roc_curve
import tensorflow as tf
import pandas as pd
import keras
import numpy as np
import itertools
import random

ext_df = pd.read_csv('/Users/eding/Desktop/U24 ROBOKOP Project/External Validation/External Validation Datasets/randomNegatives External Validation Set.csv')

model_auc_values = []

unique_protein_vectors = np.array(protein_df.iloc[:,1:201])
unique_disease_vectors = np.array(disease_df.iloc[:,1:201])
unique_drug_vectors = np.array(drug_df.iloc[:,1:201])

unique_protein_dict_names = np.array(protein_df.iloc[:,0])
unique_disease_dict_names = np.array(disease_df.iloc[:,0])
unique_drug_dict_names = np.array(drug_df.iloc[:,0])


def compute_threshold(vector_dict,n=0.5):
    distances = []
    for i in range((len(vector_dict)-1)):
        smallest_distance = 10000000
        for j in range(i+1, len(vector_dict)):
            distance = np.linalg.norm(vector_dict[i]-vector_dict[j])
            if distance < smallest_distance:
                smallest_distance = distance
        distances.append(smallest_distance)

    distances = np.array(distances)
    print(len(distances))
    print('min', np.min(distances))
    mean_distances = np.sum(distances)/len(distances)
    stdev_distance = np.std(distances)
    print('mean',mean_distances)
    print('stdev',stdev_distance)

    applicability_domain = mean_distances+(n*stdev_distance)

    return applicability_domain

drug_threshold = compute_threshold(unique_drug_vectors)
disease_threshold = compute_threshold(unique_disease_vectors)
protein_threshold = compute_threshold(unique_protein_vectors)

print(drug_threshold)
print(disease_threshold)
print(protein_threshold)

random_seeds = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,42]

for i in random_seeds:
    random.seed(i)
    positive_triples_vector_array = []
    for j in range(0,len(df_80)):
        try:
            drug_vector = drug_df.iloc[drug_dict[df_80.iloc[j,0]],1:201].tolist()
            disease_vector = disease_df.iloc[disease_dict[df_80.iloc[j,1]],1:201].tolist()
            protein_vector = protein_df.iloc[protein_dict[df_80.iloc[j,2]],1:201].tolist()
        
            drug_name = df_80.iloc[j,0]
            disease_name = df_80.iloc[j,1]
            protein_name = df_80.iloc[j,2]
        
            row = [[drug_name],[disease_name],[protein_name],drug_vector,disease_vector,protein_vector,[1]]
            merged = list(itertools.chain(*row))
            positive_triples_vector_array.append(merged)

        except KeyError:
            continue

    positive_triples_dataframe = pd.DataFrame(positive_triples_vector_array)

    random_rows = positive_triples_dataframe.sample(int(len(positive_triples_dataframe)/4), random_state = i)

    #we are now going to split these rows into 3 equal chunks. 
    #the triples in the first chunk will have its protein parameter randomized, 
    #the triples in the second chunk will have its disease parameter randomized, and so on

    split_rows = np.array_split(random_rows, 3)

    # Each part is now a separate DataFrame
    drug_disease_x, drug_x_protein, x_disease_protein = split_rows[0], split_rows[1], split_rows[2]

    drug_disease_x_negative_triples = []
    drug_x_protein_negative_triples = []
    x_disease_protein_negative_triples = []
    temp = set()
    j=0
    for j in range(len(drug_disease_x)):
        reference = np.array(drug_disease_x.iloc[j, 403:603]) ###not including 401, including 601
        reference = [float(item) for item in reference]
        counter=0
        for k in range(len(unique_protein_vectors)):
            if (np.linalg.norm(reference-unique_protein_vectors[k]) < protein_threshold):
                triple_name = [drug_disease_x.iloc[j,0], drug_disease_x.iloc[j,1], unique_protein_dict_names[k]]
                new_combo = drug_disease_x.iloc[j,0] + " " + drug_disease_x.iloc[j,1] + " " + unique_protein_dict_names[k]
                if (new_combo in pos_neg_triples_dictionary or new_combo in temp):
                    continue
                temp.add(new_combo)
                triple_vector = list(itertools.chain(*[np.array(drug_disease_x.iloc[j,3:403]).tolist(), unique_protein_vectors[k].tolist()]))
                row = list(itertools.chain(*[triple_name,triple_vector,[0]]))
                drug_disease_x_negative_triples.append(row)
                counter+=1
                if counter >=6:
                    break
    j=0
    k=0
    for j in range(len(drug_x_protein)):
        reference = np.array(drug_x_protein.iloc[j, 203:403])
        reference = [float(item) for item in reference]
        counter=0
        for k in range(len(unique_disease_vectors)):
            if (np.linalg.norm(reference-unique_disease_vectors[k]) < disease_threshold):
                triple_name = [drug_x_protein.iloc[j,0], unique_disease_dict_names[k],drug_x_protein.iloc[j,2]]
                new_combo = drug_x_protein.iloc[j,0] + " " + unique_disease_dict_names[k] + " " + drug_x_protein.iloc[j,2]
                if (new_combo in pos_neg_triples_dictionary or new_combo in temp):
                    continue
                temp.add(new_combo)
                triple_vector = list(itertools.chain(*[np.array(drug_x_protein.iloc[j,3:203]).tolist(), unique_disease_vectors[k].tolist(),np.array(drug_x_protein.iloc[j,403:603]).tolist()]))
                row = list(itertools.chain(*[triple_name,triple_vector,[0]]))
                drug_x_protein_negative_triples.append(row)
                counter +=1
                if counter >=6:
                    break
    j=0
    k=0

    for j in range(len(x_disease_protein)):
        reference = np.array(x_disease_protein.iloc[j, 3:203])
        reference = [float(item) for item in reference]
        counter=0    
        for k in range(len(unique_drug_vectors)):
            if (np.linalg.norm(reference-unique_drug_vectors[k]) < drug_threshold):
                triple_name = [unique_drug_dict_names[k],x_disease_protein.iloc[j,1], x_disease_protein.iloc[j,2]]
                new_combo = unique_drug_dict_names[k] + " " + x_disease_protein.iloc[j,1] + " " + x_disease_protein.iloc[j,2]
                if (new_combo in pos_neg_triples_dictionary or new_combo in temp):
                    continue
                temp.add(new_combo)
                triple_vector = list(itertools.chain(*[unique_drug_vectors[k].tolist(),np.array(x_disease_protein.iloc[j,203:603]).tolist()]))
                row = list(itertools.chain(*[triple_name,triple_vector,[0]]))
                x_disease_protein_negative_triples.append(row)
                counter += 1
                if counter >=5:
                    break
                
    negative_df = pd.concat([pd.DataFrame(drug_disease_x_negative_triples),pd.DataFrame(drug_x_protein_negative_triples),pd.DataFrame(x_disease_protein_negative_triples)],axis=0,ignore_index=True)
    df = pd.concat([positive_triples_dataframe,negative_df],axis=0)
    df.columns = unified_columns

    test = pd.concat([df_diverseNegs, df],axis=0)
    col = ['0', '1', '2']
    print(len(test))
    rem = test.drop_duplicates(subset = col)
    print(len(rem))

    shuffled_df = df.sample(frac=1.0, random_state=i)

    data = shuffled_df.iloc[:,3:-1].values
    labels = shuffled_df.iloc[:,-1].values

    num_folds = 5
    skf = StratifiedKFold(n_splits=num_folds, shuffle=True, random_state=i)

    all_predictions = []
    all_test_sets = []

    # Perform 5-fold cross-validation to test the accuracy of the model######
    fold = 1
    auc_values = []

    keras.utils.set_random_seed(i)
    #tf.config.experimental.enable_op_determinism()

    shuffled_df = df.sample(frac=1.0, random_state=i)

    data = shuffled_df.iloc[:,3:-1].values
    labels = shuffled_df.iloc[:,-1].values

    num_folds = 5
    skf = StratifiedKFold(n_splits=num_folds, shuffle=True, random_state=i)

    all_predictions = []
    all_test_sets = []

    # Perform 5-fold cross-validation to test the accuracy of the model######
    fold = 1
    auc_values = []

    keras.utils.set_random_seed(i)
    #tf.config.experimental.enable_op_determinism()

    for train_index, test_index in skf.split(data, labels):
        print(f"Fold: {fold}")
        print(train_index)
    
        # Split the data into training and test sets
        X_train, X_test = data[train_index], data[test_index]
        y_train, y_test = labels[train_index], labels[test_index]
        tf_train_set = tf.data.Dataset.from_tensor_slices((X_train,y_train))
        tf_test_set = tf.data.Dataset.from_tensor_slices((X_test,y_test))
        
        BATCH_SIZE = 32
        STEPS_PER_EPOCH = len(X_train)//BATCH_SIZE
        tf_train_set = tf_train_set.repeat().batch(BATCH_SIZE)
        tf_test_set = tf_test_set.batch(BATCH_SIZE)

    # Define the learning rate schedule
        lr_schedule = tf.keras.optimizers.schedules.InverseTimeDecay(
            0.001,
            decay_steps=STEPS_PER_EPOCH * 1000,
            decay_rate=1,
            staircase=False
        )
        
        def get_optimizer():
            return tf.keras.optimizers.Adam(lr_schedule)

        optimizer = get_optimizer()
        
        model = tf.keras.models.Sequential([
            tf.keras.Input(shape=(600,)),
            tf.keras.layers.Dense(1024, activation='relu',kernel_regularizer=tf.keras.regularizers.l2(0.001)), tf.keras.layers.BatchNormalization(),
            tf.keras.layers.Dropout(0.5),
            tf.keras.layers.Dense(512, activation='relu',kernel_regularizer=tf.keras.regularizers.l2(0.001)),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.Dropout(0.5),
            tf.keras.layers.Dense(256, activation='relu',kernel_regularizer=tf.keras.regularizers.l2(0.001)),
            tf.keras.layers.Dropout(0.5),
            tf.keras.layers.Dense(1, activation='sigmoid')
        ])

        model.compile(optimizer=optimizer,
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

        # Implement early stopping
        early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
        # Train the model
        model.fit(tf_train_set,
                        steps_per_epoch=STEPS_PER_EPOCH, 
                        epochs=100,
                        validation_data = tf_test_set, 
                        callbacks=[early_stopping], 
                        verbose=0)
        
        predictions = model.predict(X_test)
        fpr,tpr, thresholds = roc_curve(y_test,predictions)
        roc_auc = auc(fpr, tpr)
        auc_values.append(roc_auc)
        print("AUROC for test data:", roc_auc)
        all_predictions.append(predictions)
        all_test_sets.append(y_test)
        
        ####Before each iteration ends, save each model###
        
        save_model= tf.keras.models.save_model(model, os.path.join(os.getcwd(), f'Classification Models/Complete Set 2 Models/ROBOMechDB Complete Set 2 Model Seed {i}.keras'))

        fold += 1

    fcv_mean_auc = np.mean(auc_values)
    fcv_std_auc = np.std(auc_values)
    fcv_min_auc = np.min(auc_values)
    fcv_max_auc = np.max(auc_values)
    print(fcv_mean_auc, fcv_min_auc,fcv_max_auc)
    model_auc_values.append([fcv_min_auc,fcv_max_auc,fcv_mean_auc,fcv_std_auc])
    
fcv_df = pd.DataFrame(model_auc_values, columns=['min_auc', 'max_auc', fcv_mean_auc,fcv_std_auc])

min_auc = np.min(fcv_df.iloc[:,0])
max_auc = np.max(fcv_df.iloc[:,1])
avg_auc = np.mean(fcv_df.iloc[:,2])
std_dev_auc = np.mean(fcv_df.iloc[:,3])

ensemble_stats = np.array([min_auc,max_auc,avg_auc,std_dev_auc])
ensemble_stats = ensemble_stats.reshape(1, -1)
ensemble_stats_df = pd.DataFrame(ensemble_stats, columns = ['min_auc', 'max_auc', 'avg_auc', 'std_dev_auc'])
ensemble_stats_df.to_csv(os.path.join(os.getcwd(), 'Classification Models/Complete Set 2 Models/complete_set_2_ensemble_stats.csv'))

998
min 1.3490766481180738
mean 3.9296513978117704
stdev 1.1837539597778333
589
min 1.3756730714527872
mean 6.979102681442049
stdev 2.672071221220308
835
min 1.00340273053115
mean 6.690955269076238
stdev 4.105268395143029
4.521528377700687
8.315138292052202
8.743589466647753


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


34588
34588
Fold: 1
[    0     1     2 ... 14433 14434 14435]
91/91 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
AUROC for test data: 0.7482234442514335
Fold: 2
[    0     3     4 ... 14433 14434 14435]
91/91 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
AUROC for test data: 0.7421449015430318
Fold: 3
[    1     2     3 ... 14432 14434 14435]
91/91 ━━━━━━━━━━━━━━━━━━━━ 0s 983us/step
AUROC for test data: 0.7419264709977043
Fold: 4
[    0     1     2 ... 14431 14432 14433]
91/91 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
AUROC for test data: 0.7397594537351206
Fold: 5
[    0     1     2 ... 14433 14434 14435]
91/91 ━━━━━━━━━━━━━━━━━━━━ 0s 976us/step
AUROC for test data: 0.7430367933821453
0.7430182127818872 0.7397594537351206 0.7482234442514335


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


34568
34568
Fold: 1
[    0     1     2 ... 14413 14414 14415]
91/91 ━━━━━━━━━━━━━━━━━━━━ 0s 922us/step
AUROC for test data: 0.7543282009588023
Fold: 2
[    0     1     2 ... 14413 14414 14415]
91/91 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step  
AUROC for test data: 0.7688578966118178
Fold: 3
[    1     3     4 ... 14413 14414 14415]
91/91 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
AUROC for test data: 0.7399216260193913
Fold: 4
[    0     2     3 ... 14412 14414 14415]
91/91 ━━━━━━━━━━━━━━━━━━━━ 0s 952us/step
AUROC for test data: 0.7672005992780547
Fold: 5
[    0     1     2 ... 14410 14412 14413]
91/91 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step  
AUROC for test data: 0.7675087113884976
0.7595634068513127 0.7399216260193913 0.7688578966118178


...

34588
34588
Fold: 1
[    0     1     4 ... 14431 14432 14435]
91/91 ━━━━━━━━━━━━━━━━━━━━ 0s 854us/step
AUROC for test data: 0.7573294168272557
Fold: 2
[    2     3     4 ... 14433 14434 14435]
91/91 ━━━━━━━━━━━━━━━━━━━━ 0s 907us/step
AUROC for test data: 0.7129155341082898
Fold: 3
[    0     1     2 ... 14433 14434 14435]
91/91 ━━━━━━━━━━━━━━━━━━━━ 0s 962us/step
AUROC for test data: 0.7123893805309734
Fold: 4
[    0     1     2 ... 14432 14433 14434]
91/91 ━━━━━━━━━━━━━━━━━━━━ 0s 857us/step
AUROC for test data: 0.729262077719083
Fold: 5
[    0     1     2 ... 14433 14434 14435]
91/91 ━━━━━━━━━━━━━━━━━━━━ 0s 912us/step
AUROC for test data: 0.7371923563625611
0.7298177531096326 0.7123893805309734 0.7573294168272557


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


34548
34548
Fold: 1
[    3     4     5 ... 14393 14394 14395]
90/90 ━━━━━━━━━━━━━━━━━━━━ 0s 632us/step
AUROC for test data: 0.7538826269720695
Fold: 2
[    0     1     2 ... 14392 14393 14395]
90/90 ━━━━━━━━━━━━━━━━━━━━ 0s 982us/step
AUROC for test data: 0.7355461274493711
Fold: 3
[    0     1     2 ... 14392 14394 14395]
90/90 ━━━━━━━━━━━━━━━━━━━━ 0s 817us/step
AUROC for test data: 0.7387039744166601
Fold: 4
[    0     1     2 ... 14393 14394 14395]
90/90 ━━━━━━━━━━━━━━━━━━━━ 0s 923us/step
AUROC for test data: 0.754448435516031
Fold: 5
[    0     1     2 ... 14392 14393 14394]
90/90 ━━━━━━━━━━━━━━━━━━━━ 0s 866us/step
AUROC for test data: 0.7612396139603821
0.7487641556629028 0.7355461274493711 0.7612396139603821


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


34561
34561
Fold: 1
[    0     2     3 ... 14405 14406 14408]
91/91 ━━━━━━━━━━━━━━━━━━━━ 0s 903us/step
AUROC for test data: 0.7624956821732651
Fold: 2
[    0     1     2 ... 14406 14407 14408]
91/91 ━━━━━━━━━━━━━━━━━━━━ 0s 882us/step
AUROC for test data: 0.7253570728290304
Fold: 3
[    0     1     2 ... 14406 14407 14408]
91/91 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step  
AUROC for test data: 0.7421295111955164
Fold: 4
[    1     2     4 ... 14405 14407 14408]
91/91 ━━━━━━━━━━━━━━━━━━━━ 0s 861us/step
AUROC for test data: 0.7491771656729707
Fold: 5
[    0     1     3 ... 14405 14406 14407]
91/91 ━━━━━━━━━━━━━━━━━━━━ 0s 929us/step
AUROC for test data: 0.7254753929548703
0.7409269649651307 0.7253570728290304 0.7624956821732651


In [5]:
print(ensemble_stats_df)

    min_auc   max_auc   avg_auc  std_dev_auc
0  0.668332  0.787279  0.737051     0.012957
